In [46]:
import operator
from functools import reduce
import math
from decimal import Decimal, getcontext

class Vector(object):
    
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = tuple([Decimal(x) for x in coordinates])
            self.dimension = len(coordinates)
        except ValueError:
            raise ValueError('The coordinates must be nonempty')
        except TypeError:
            raise TypeError('The coordinates must be an iterable')
            
    def __str__(self):
        return 'Vector: {}'.format(tuple(round(x,3) for x in self.coordinates))
    
    def __eq__(self, v):
        return self.coordinates == v.coordinates
    
    def __add__(self, v):
        return Vector(tuple(map(operator.add, self.coordinates, v.coordinates)))
    
    def __sub__(self, v):
        return Vector(tuple(map(operator.sub, self.coordinates, v.coordinates)))
    
    def __mul__(self, v):
        return Vector(tuple(map((Decimal(v)).__mul__, self.coordinates)))
    
    def magnitude(self):
        return Decimal(math.pow(reduce(lambda x,y: x + math.pow(y,2), self.coordinates, 0), 0.5))
    
    def normalized(self):
        mag = self.magnitude()
        if (mag == 0):
            raise Exception('Cannot normalize the zero vector')
        return self * (Decimal(1.0) / mag)
    
    def dot(self, v):
        if (v.dimension != self.dimension):
            raise Exception('Unequal dimensions {0} & {1}'.format(self.dimension, v.dimension))
        return reduce(lambda x, y: x + y, [x * y for (x,y) in zip(self.coordinates, v.coordinates)], 0)
    
    def cosAngle(self, v):
        dotproduct = self.dot(v)
        selfmag = self.magnitude()
        othermag = v.magnitude()
        # print(type(dotproduct), type(selfmag), type(othermag))
        try:
            return dotproduct / (selfmag * othermag)
        except ZeroDivisionError:
            raise Exception('Cannot get angle with zero vector')
            
    def angleRadians(self, v):
        return math.acos(self.cosAngle(v))
    
    def angleDegrees(self, v):
        return self.angleRadians(v) * 180 / math.pi
    
    def is_zero(self, tolerance = 1e-10):
        return self.magnitude() < tolerance
    
    def isParallel(self, v, tolerance = 1e-10):
        if (self.is_zero() or v.is_zero()):
            return True
        absangle = abs(self.cosAngle(v))
        # print('absangle = {}'.format(absangle))
        return abs(absangle - 1) < tolerance
    
    def isOrthogonal(self, v, tolerance = 1e-10):
        dotproduct = self.dot(v)
        # print('dot product is {}'.format(dotproduct))
        return abs(dotproduct) < tolerance
    
    def component_parallel_to(self, basis):
        '''
        Project this vector onto v (v is the basis vector) and get the parallel component
        '''
        if (basis.is_zero()):
            raise Exception('Cannot project to zero vector')
        norm = basis.normalized()
        return norm * self.dot(norm)
    
    def component_orthogonal_to(self, basis):
        return self - self.component_parallel_to(basis)

    def cross_product(self, v):
        '''
        cross product of v and w = (magnitude of v) * (magnitude of w) * (sin(theta))
        '''
        # sinAngle = math.sqrt(1 - math.pow(self.cosAngle(v), 2))
        # weight = self.magnitude() * v.magnitude()
        # mag_cross = weigth * sinAngle
        if (self.dimension != 3 and v.dimension != 3):
            raise Exception("You can only do cross product of 3 dimensional vectors")
        return Vector([
                self.coordinates[1] * v.coordinates[2] - v.coordinates[1] * self.coordinates[2],
                - (self.coordinates[0] * v.coordinates[2] - v.coordinates[0] * self.coordinates[2]),
                self.coordinates[0] * v.coordinates[1] - v.coordinates[0] * self.coordinates[1],
                      ])   
    def area_parallelogram(self, v):
        cp = self.cross_product(v)
        return cp.magnitude()
    
    def area_triangle(self, v):
        cp = self.cross_product(v)
        return Decimal(0.5) * cp.magnitude()

def print3dec(val):
    print(round(val, 3))

my_vector = Vector([1,2,3])
my_vector2 = Vector([1,2,3])
my_vector3 = Vector([-11,2,3])
print(my_vector)
print(my_vector == my_vector2)
print(my_vector == my_vector3)
print(my_vector + my_vector2)
print(my_vector - my_vector2)
print(my_vector * 3)

Vector: (1.0, 2.0, 3.0)
True
False
Vector: (2.0, 4.0, 6.0)
Vector: (0.0, 0.0, 0.0)
Vector: (3.0, 6.0, 9.0)


In [74]:
v1 = Vector([8.218, -9.341])
v2 = Vector([-1.129, 2.111])
print(v1 + v2)
v1 = Vector([7.119, 8.215])
v2 = Vector([-8.223, 0.878])
print(v1 - v2)
v3 = Vector([1.671, -1.012, -0.318])
print(v3 * 7.41)

Vector: (7.089, -7.23)
Vector: (15.342, 7.337)
Vector: (12.382, -7.499, -2.356)


In [75]:
# math.pow(reduce(lambda x,y: x+y, [1,2,3], 0), 0.5)
print(round(Vector([-0.221, 7.437]).magnitude(), 3))
print(round(Vector([8.813, -1.331, -6.247]).magnitude(), 3))
print(Vector([5.581, -2.136]).normalized())
print(Vector([1.996, 3.108, -4.554]).normalized())
# Vector([0, 0, 0]).norm()

7.44
10.884
Vector: (0.934, -0.357)
Vector: (0.34, 0.53, -0.777)


In [109]:
v = Vector([7.887, 4.138])
w = Vector([-8.802, 6.776])
print3dec(v.dot(w))

v = Vector([-5.955, -4.904, -1.874])
w = Vector([-4.496, -8.755, 7.103])
print3dec(v.dot(w))

v = Vector([3.183, -7.627])
w = Vector([-2.668, 5.319])
print3dec(v.angleRadians(w))

v = Vector([7.35, 0.221, 5.188])
w = Vector([2.751, 8.259, 3.985])
print3dec(v.angleDegrees(w))



-41.382
56.397
3.072
60.276


In [2]:
v = Vector([-7.579, -7.88])
w = Vector([22.737, 23.64])
print(v.isParallel(w), v.isOrthogonal(w))

v = Vector([-2.029, 9.97, 4.172])
w = Vector([-9.231, -6.639, -7.245])
print(v.isParallel(w), v.isOrthogonal(w))

v = Vector([-2.328, -7.284, -1.214])
w = Vector([-1.821, 1.072, -2.94])
print(v.isParallel(w), v.isOrthogonal(w))

v = Vector([2.118, 4.827])
w = Vector([0, 0])
print(v.isParallel(w), v.isOrthogonal(w))

(True, False)
(False, False)
(False, True)
(True, True)


In [14]:
v1 = Vector([3,0,0])
print(my_vector)
print(v1)
print(v1.dot(my_vector))
print(my_vector.project(v1))

Vector: (1.0, 2.0, 3.0)
Vector: (3.0, 0.0, 0.0)
3
Vector: (1.0, 0.0, 0.0)


In [34]:
v = Vector([3.039, 1.879])
b = Vector([0.825, 2.036])
print(v.component_parallel_to(b))

v = Vector([-9.88, -3.264, -8.159])
b = Vector([-2.155, -9.353, -9.473])
print(v.component_orthogonal_to(b))

v = Vector([3.009, -6.172, 3.692, -2.51])
b = Vector([6.404, -9.144, 2.759, 8.718])
proj = v.component_parallel_to(b)
orth = v.component_orthogonal_to(b)
print(proj)
print(orth)
print(proj + orth)

Vector: (1.083, 2.672)
Vector: (-8.35, 3.376, -1.434)
Vector: (1.969, -2.811, 0.848, 2.68)
Vector: (1.04, -3.361, 2.844, -5.19)
Vector: (3.009, -6.172, 3.692, -2.51)


In [47]:
v = Vector([5,3,-2])
w = Vector([-1,0,3])
print(v.cross_product(w))

v = Vector([8.462, 7.893, -8.187])
w = Vector([6.984, -5.975, 4.778])
print(v.cross_product(w))

v = Vector([-8.987, -9.838, 5.031])
w = Vector([-4.268, -1.861, -8.866])
print3dec(v.area_parallelogram(w))

v = Vector([1.5, 9.547, 3.691])
w = Vector([-6.007, 0.124, 5.772])
print3dec(v.area_triangle(w))

Vector: (9.0, -13.0, 3.0)
Vector: (-11.205, -97.609, -105.685)
142.122
42.565
